# Chapter 50: Compliance and Governance

Modern software delivery operates within a complex regulatory landscape where velocity must coexist with accountability. Compliance is not a checkbox exercise performed annually; it is a continuous state of evidence that systems meet security, privacy, and operational standards. This chapter transforms compliance from a manual audit burden into an automated, verifiable property of your CI/CD pipeline. We examine major regulatory frameworks—**SOC 2**, **PCI-DSS**, **HIPAA**, and **GDPR**—and their specific implications for containerized workloads. We implement **audit trails** that capture every state change, **policy enforcement** that prevents non-compliant resources from reaching production, **approval workflows** that enforce segregation of duties, and **automated reporting** that provides auditors with real-time evidence of control effectiveness. These practices ensure that your platform satisfies both regulatory requirements and internal governance without sacrificing deployment velocity.

## 50.1 Regulatory Requirements

Understanding the regulatory landscape is essential for designing compliant systems. Each framework imposes specific technical controls on CI/CD pipelines.

### SOC 2 (Service Organization Control 2)

**Scope**: Security, availability, processing integrity, confidentiality, and privacy of customer data.

**CI/CD Implications**:
- **CC6.1** (Logical access): Role-based access to pipelines and production
- **CC6.6** (Encryption): Encryption of artifacts in transit and at rest
- **CC7.2** (System monitoring): Audit logs of all deployments
- **A1.2** (Availability): Change management and testing requirements

**Implementation**:
```yaml
# SOC 2 compliant pipeline gate
- name: SOC2 Compliance Gate
  run: |
    # Verify change ticket exists
    if ! curl -H "Authorization: Bearer ${{ secrets.ITSM_TOKEN }}" \
         "https://itsm.company.com/api/change/${{ github.event.inputs.change_ticket }}" | \
         jq -e '.status == "approved"'; then
      echo "SOC 2 CC7.2: Approved change ticket required"
      exit 1
    fi
    
    # Verify two-person approval
    APPROVERS=$(curl -H "Authorization: token ${{ secrets.GITHUB_TOKEN }}" \
      "https://api.github.com/repos/${{ github.repository }}/pulls/${{ github.event.pull_request.number }}/reviews" | \
      jq '[.[] | select(.state == "APPROVED")] | length')
    
    if [ "$APPROVERS" -lt 2 ]; then
      echo "SOC 2 CC6.1: Two-person approval required for production"
      exit 1
    fi
```

### PCI-DSS (Payment Card Industry Data Security Standard)

**Scope**: Any entity storing, processing, or transmitting cardholder data.

**Key Requirements**:
- **Req 6.5**: Address common coding vulnerabilities (OWASP Top 10)
- **Req 8.2**: Strong authentication (MFA for production access)
- **Req 10.2**: Audit trails for all access to cardholder data
- **Req 11.2.2**: Vulnerability scanning quarterly

**Container-Specific Controls**:
```yaml
# PCI-DSS compliant deployment
apiVersion: apps/v1
kind: Deployment
metadata:
  name: payment-service
spec:
  template:
    spec:
      containers:
      - name: app
        image: payment-service:latest
        securityContext:
          readOnlyRootFilesystem: true  # Req 6.5: Immutable filesystem
          runAsNonRoot: true
          allowPrivilegeEscalation: false
          capabilities:
            drop:
            - ALL
        resources:
          limits:
            cpu: "1000m"
            memory: "1Gi"
        env:
        - name: DATABASE_PASSWORD
          valueFrom:
            secretKeyRef:
              name: db-credentials
              key: password
        volumeMounts:
        - name: tmp
          mountPath: /tmp
        - name: audit-logs
          mountPath: /var/log/audit
      volumes:
      - name: tmp
        emptyDir: {}
      - name: audit-logs
        hostPath:
          path: /var/log/payment-service
          type: DirectoryOrCreate
      # PCI Req 10.2: Audit logging sidecar
      - name: audit-sidecar
        image: fluentd
        volumeMounts:
        - name: audit-logs
          mountPath: /var/log/input
```

### HIPAA (Health Insurance Portability and Accountability Act)

**Scope**: Protected Health Information (PHI) handling.

**Technical Safeguards**:
- **§164.312(a)(1)**: Access Control (unique user IDs, emergency access)
- **§164.312(a)(2)(iv)**: Encryption and Decryption
- **§164.312(b)**: Audit Controls (logging access to PHI)
- **§164.312(c)(1)**: Integrity (protection against improper alteration)

**Implementation**:
```yaml
# HIPAA-compliant audit logging
apiVersion: v1
kind: ConfigMap
metadata:
  name: hipaa-audit-config
data:
  audit-policy.yaml: |
    apiVersion: audit.k8s.io/v1
    kind: Policy
    rules:
    # Log all requests to PHI-containing resources
    - level: RequestResponse
      resources:
      - group: ""
        resources: ["pods"]
      namespaces: ["phi-data"]
      omitStages:
      - RequestReceived
      
    # Log authentication attempts
    - level: Metadata
      resources:
      - group: "authentication.k8s.io"
        resources: ["tokenreviews"]
        
    # Default: don't log system components
    - level: None
      userGroups: ["system:serviceaccounts:kube-system"]
```

### GDPR (General Data Protection Regulation)

**Scope**: Personal data of EU residents.

**Key Requirements**:
- **Article 25**: Data protection by design and default
- **Article 30**: Records of processing activities (audit trails)
- **Article 32**: Security of processing (encryption, resilience)
- **Article 33**: Breach notification (72-hour reporting)

**Data Minimization in CI/CD**:
```yaml
# Avoid logging PII in pipeline
- name: Run Tests
  run: |
    # Sanitize logs
    pytest tests/ 2>&1 | sed 's/email=[^ ]*/email=REDACTED/g' | \
                         sed 's/ssn=[0-9-]*/ssn=REDACTED/g'
  
- name: Upload Artifacts
  uses: actions/upload-artifact@v3
  with:
    name: test-results
    path: results.xml
    # Ensure no PII in artifacts
    if-no-files-found: error
```

## 50.2 Audit Trails

Compliance requires comprehensive, immutable records of who did what, when, and why. Audit trails must capture pipeline executions, deployments, and access to sensitive resources.

### Kubernetes Audit Logging

**Audit Policy Configuration**:
```yaml
apiVersion: audit.k8s.io/v1
kind: Policy
rules:
  # Log all mutations to production deployments
  - level: RequestResponse
    resources:
    - group: "apps"
      resources: ["deployments"]
    namespaces: ["production"]
    verbs: ["create", "update", "patch", "delete"]
    
  # Log secret access
  - level: RequestResponse
    resources:
    - group: ""
      resources: ["secrets"]
    verbs: ["get", "list", "watch"]
    
  # Log authentication
  - level: Metadata
    resources:
    - group: "authentication.k8s.io"
      resources: ["tokenreviews", "subjectaccessreviews"]
      
  # Ignore system noise
  - level: None
    userGroups: ["system:serviceaccounts:kube-system"]
    resources:
    - group: ""
      resources: ["endpoints", "configmaps"]
```

**Audit Backend Configuration** (API Server flags):
```bash
kube-apiserver \
  --audit-log-path=/var/log/audit/audit.log \
  --audit-log-maxage=30 \
  --audit-log-maxbackup=10 \
  --audit-log-maxsize=100 \
  --audit-policy-file=/etc/kubernetes/audit-policy.yaml \
  --audit-webhook-config-file=/etc/kubernetes/audit-webhook.yaml \
  --audit-webhook-mode=batch
```

**Webhook Backend** (for SIEM integration):
```yaml
# audit-webhook.yaml
apiVersion: v1
kind: Config
clusters:
- name: siem-cluster
  cluster:
    server: https://siem.company.com/k8s-audit
    certificate-authority: /etc/kubernetes/siem-ca.crt
contexts:
- context:
    cluster: siem-cluster
    user: k8s-audit
  name: audit-context
current-context: audit-context
users:
- name: k8s-audit
  user:
    token: ${AUDIT_WEBHOOK_TOKEN}
```

### CI/CD Audit Trails

**GitHub Actions**:
```yaml
- name: Generate Audit Log
  run: |
    cat > audit-$(date +%s).json <<EOF
    {
      "timestamp": "$(date -u +%Y-%m-%dT%H:%M:%SZ)",
      "actor": "${{ github.actor }}",
      "event": "${{ github.event_name }}",
      "sha": "${{ github.sha }}",
      "repository": "${{ github.repository }}",
      "workflow": "${{ github.workflow }}",
      "job": "${{ github.job }}",
      "runner": "${{ runner.name }}",
      "ip_address": "${{ runner.ip }}",
      "approvers": "${{ github.event.inputs.approvers }}"
    }
    EOF
    
- name: Upload to SIEM
  run: |
    curl -X POST https://siem.company.com/api/v1/events \
      -H "Authorization: Bearer ${{ secrets.SIEM_TOKEN }}" \
      -H "Content-Type: application/json" \
      -d @audit-*.json
```

**GitLab CI**:
```yaml
audit:compliance:
  stage: .post
  script:
    - |
      cat > compliance.json <<EOF
      {
        "pipeline_id": "$CI_PIPELINE_ID",
        "project": "$CI_PROJECT_PATH",
        "commit_sha": "$CI_COMMIT_SHA",
        "user": "$GITLAB_USER_LOGIN",
        "timestamp": "$(date -u +%Y-%m-%dT%H:%M:%SZ)",
        "artifacts": {
          "sbom": "$CI_JOB_URL/artifacts/raw/sbom.json",
          "scan_results": "$CI_JOB_URL/artifacts/raw/scan-results.json"
        }
      }
      EOF
    - aws s3 cp compliance.json s3://compliance-bucket/$CI_COMMIT_SHA/
  only:
    - main
```

### Immutable Audit Logs

Ensure logs cannot be tampered with:

**WORM Storage** (AWS S3 Object Lock):
```bash
# Create compliance bucket with legal hold
aws s3api create-bucket \
  --bucket k8s-audit-logs \
  --region us-east-1 \
  --object-lock-enabled-for-bucket

aws s3api put-object-lock-configuration \
  --bucket k8s-audit-logs \
  --object-lock-configuration '{
    "ObjectLockEnabled": "Enabled",
    "Rule": {
      "DefaultRetention": {
        "Mode": "COMPLIANCE",
        "Years": 7
      }
    }
  }'
```

**Blockchain Anchoring** (Advanced):
```python
# Hash audit logs and anchor to blockchain for immutability proof
import hashlib
import json
from web3 import Web3

def anchor_to_blockchain(audit_log):
    log_hash = hashlib.sha256(json.dumps(audit_log).encode()).hexdigest()
    
    w3 = Web3(Web3.HTTPProvider('https://ethereum-node.company.com'))
    contract = w3.eth.contract(address=ANCHOR_CONTRACT, abi=ABI)
    
    tx_hash = contract.functions.anchorHash(log_hash).transact({
        'from': w3.eth.accounts[0]
    })
    
    return tx_hash
```

## 50.3 Policy Enforcement

Manual compliance checks are insufficient at scale. Policy-as-code automates enforcement of organizational standards.

### Open Policy Agent (OPA) / Gatekeeper

**Constraint Templates** (Reusable policies):
```yaml
apiVersion: templates.gatekeeper.sh/v1
kind: ConstraintTemplate
metadata:
  name: k8srequiredlabels
spec:
  crd:
    spec:
      names:
        kind: K8sRequiredLabels
      validation:
        openAPIV3Schema:
          properties:
            labels:
              type: array
              items:
                type: string
  targets:
    - target: admission.k8s.gatekeeper.sh
      rego: |
        package k8srequiredlabels

        violation[{"msg": msg}] {
          provided := {label | input.review.object.metadata.labels[label]}
          required := {label | label := input.parameters.labels[_]}
          missing := required - provided
          count(missing) > 0
          msg := sprintf("Missing required labels: %v", [missing])
        }
```

**Applying Constraints**:
```yaml
apiVersion: constraints.gatekeeper.sh/v1beta1
kind: K8sRequiredLabels
metadata:
  name: require-compliance-labels
spec:
  match:
    kinds:
      - apiGroups: [""]
        kinds: ["Namespace"]
  parameters:
    labels:
      - "compliance.company.com/owner"
      - "compliance.company.com/data-classification"
      - "compliance.company.com/cost-center"
```

### Kyverno (Kubernetes-Native Policy)

Simpler syntax than OPA, native to Kubernetes:

**Require Non-Root**:
```yaml
apiVersion: kyverno.io/v1
kind: ClusterPolicy
metadata:
  name: require-non-root
spec:
  validationFailureAction: Enforce
  background: true
  rules:
  - name: check-run-as-non-root
    match:
      resources:
        kinds:
        - Pod
    validate:
      message: "Running as root is not allowed"
      pattern:
        spec:
          securityContext:
            runAsNonRoot: true
          containers:
          - securityContext:
              allowPrivilegeEscalation: false
```

**Require Resource Limits** (Prevent DoS/Noisy Neighbor):
```yaml
apiVersion: kyverno.io/v1
kind: ClusterPolicy
metadata:
  name: require-resources
spec:
  validationFailureAction: Enforce
  rules:
  - name: check-memory-limits
    match:
      resources:
        kinds:
        - Deployment
        - StatefulSet
    validate:
      message: "Memory limits and requests are required"
      pattern:
        spec:
          template:
            spec:
              containers:
              - resources:
                  limits:
                    memory: "?*"
                  requests:
                    memory: "?*"
```

**Auto-Generate Network Policies**:
```yaml
apiVersion: kyverno.io/v1
kind: ClusterPolicy
metadata:
  name: auto-network-policy
spec:
  rules:
  - name: generate-default-deny
    match:
      resources:
        kinds:
        - Namespace
    generate:
      kind: NetworkPolicy
      name: default-deny-all
      namespace: "{{request.object.metadata.name}}"
      data:
        spec:
          podSelector: {}
          policyTypes:
          - Ingress
          - Egress
```

### Policy Testing

**Conftest** (OPA policies in CI):
```bash
# Test Kubernetes manifests against policies
conftest test deployment.yaml --policy policies/

# In CI
- name: Policy Check
  run: |
    conftest test k8s-manifests/ --policy policies/ --output junit > policy-results.xml
```

**Kyverno CLI**:
```bash
# Test policy against resource
kyverno test . --test-file test.yaml

# test.yaml
name: test-require-labels
policies:
- require-labels.yaml
resources:
- resource.yaml
results:
- policy: require-labels
  rule: check-labels
  resource: myapp
  kind: Deployment
  result: pass
```

## 50.4 Approval Workflows

Segregation of duties requires that deployment to production cannot be performed by the same person who wrote the code or triggered the build.

### GitHub Environments

**Environment Protection Rules**:
```yaml
# .github/workflows/deploy.yml
jobs:
  deploy-production:
    runs-on: ubuntu-latest
    environment:
      name: production
      url: https://app.company.com
    steps:
      - name: Deploy
        run: kubectl apply -f k8s/production/
```

**Required Reviewers** (Configured in UI or API):
- Minimum 2 approvers from `platform-engineering` team
- No self-approval
- Required status checks must pass

### GitLab Protected Environments

```yaml
deploy:production:
  stage: deploy
  script:
    - kubectl apply -f k8s/production/
  environment:
    name: production
    url: https://app.company.com
    deployment_tier: production
  only:
    - main
  when: manual  # Require button click
  allow_failure: false
  needs:
    - job: test
      artifacts: true
    - job: security-scan
      artifacts: true
```

**Approval Rules**:
```yaml
# .gitlab/approval_rules.yml
approval_rules:
  - name: Production Deployment
    applies_to: 
      - deploy:production
    approvers:
      - user: lead-engineer
      - group: platform-team
    approvals_required: 2
    prevent_self_approval: true
```

### Policy-Based Approvals (OPA)

Complex approval logic based on change scope:
```rego
package pipeline.approval

import rego.v1

# Require approval for production if:
# 1. Database migrations included, OR
# 2. More than 10 files changed, OR
# 3. Critical path modified

default allow := false

allow if {
  input.environment != "production"
}

allow if {
  input.environment == "production"
  input.approvals.count >= 2
  input.approvals.seniority >= "senior"
  not input.changes.contains_database_migration
}

allow if {
  input.environment == "production"
  input.approvals.count >= 3
  input.changes.contains_database_migration
  input.approvals.includes_dba
}
```

## 50.5 Change Management

Integration with IT Service Management (ITSM) tools ensures traceability from business request to deployed code.

### ServiceNow Integration

**Create Change Request from Pipeline**:
```yaml
- name: Create Change Request
  id: create-cr
  run: |
    CR_RESPONSE=$(curl -X POST \
      -H "Authorization: Bearer ${{ secrets.SNOW_TOKEN }}" \
      -H "Content-Type: application/json" \
      "https://company.service-now.com/api/now/table/change_request" \
      -d '{
        "short_description": "Deploy ${{ github.sha }} to production",
        "description": "Automated deployment from GitHub Actions\nCommit: ${{ github.sha }}\nBranch: ${{ github.ref }}\nAuthor: ${{ github.actor }}",
        "category": "Software",
        "risk": "Moderate",
        "impact": "2",  # Medium
        "urgency": "2", # Medium
        "start_date": "'$(date -u +%Y-%m-%d)'",
        "end_date": "'$(date -u -d "+1 hour" +%Y-%m-%d)'"
      }')
    
    CR_NUMBER=$(echo $CR_RESPONSE | jq -r '.result.number')
    echo "cr_number=$CR_NUMBER" >> $GITHUB_OUTPUT

- name: Update CR with Deployment Status
  if: always()
  run: |
    STATE=$(if [ "${{ job.status }}" == "success" ]; then echo "3"; else echo "4"; fi)  # 3=Successful, 4=Unsuccessful
    
    curl -X PATCH \
      -H "Authorization: Bearer ${{ secrets.SNOW_TOKEN }}" \
      "https://company.service-now.com/api/now/table/change_request/${{ steps.create-cr.outputs.cr_number }}" \
      -d "{
        \"state\": \"$STATE\",
        \"close_code\": \"Successful\",
        \"close_notes\": \"Deployment ${{ job.status }}. Pipeline: ${{ github.run_id }}\"
      }"
```

### Jira Integration

**Link Deployments to Tickets**:
```yaml
- name: Verify Ticket Status
  run: |
    TICKET=$(echo "${{ github.event.head_commit.message }}" | grep -oE '[A-Z]+-[0-9]+' | head -1)
    
    if [ -z "$TICKET" ]; then
      echo "ERROR: Commit message must include Jira ticket (e.g., PROJ-123)"
      exit 1
    fi
    
    STATUS=$(curl -H "Authorization: Bearer ${{ secrets.JIRA_TOKEN }}" \
      "https://company.atlassian.net/rest/api/3/issue/$TICKET" | \
      jq -r '.fields.status.name')
    
    if [ "$STATUS" != "Ready for Deployment" ]; then
      echo "Ticket $TICKET status is '$STATUS', not 'Ready for Deployment'"
      exit 1
    fi
    
    echo "ticket=$TICKET" >> $GITHUB_OUTPUT

- name: Update Jira After Deployment
  if: success()
  run: |
    curl -X POST \
      -H "Authorization: Bearer ${{ secrets.JIRA_TOKEN }}" \
      -H "Content-Type: application/json" \
      "https://company.atlassian.net/rest/api/3/issue/${{ steps.verify-ticket.outputs.ticket }}/transitions" \
      -d '{
        "transition": {"id": "31"},
        "update": {
          "comment": [{"add": {"body": "Deployed to production via pipeline ${{ github.run_id }}"}}]
        }
      }'
```

## 50.6 Documentation Requirements

Compliance requires evidence that controls are implemented and effective. Documentation must be generated automatically from code, not maintained manually.

### Automated Documentation

**Architecture Decision Records (ADRs)** in Git:
```markdown
# 0001-use-external-secrets-operator.md

## Status
Accepted

## Context
We need to manage secrets in Kubernetes without storing them in Git.

## Decision
Use External Secrets Operator with AWS Secrets Manager.

## Consequences
- Secrets are stored in cloud KMS
- Automatic rotation possible
- Dependency on AWS availability
```

**Living Documentation from Code**:
```yaml
# Generate compliance documentation from Terraform
- name: Generate Compliance Docs
  run: |
    terraform-docs markdown . > INFRASTRUCTURE.md
    
    # Extract security groups and rules
    terraform show -json | jq '
      .values.root_module.resources[] | 
      select(.type == "aws_security_group") |
      {
        name: .values.name,
        rules: .values.ingress
      }
    ' > SECURITY_RULES.json

- name: Update Wiki
  run: |
    # Push to Confluence or GitHub Wiki
    python scripts/update_docs.py INFRASTRUCTURE.md
```

### Evidence Collection

**Compliance Evidence Artifacts**:
```yaml
# .github/workflows/compliance.yml
name: Compliance Evidence
on:
  push:
    branches: [main]
  schedule:
    - cron: '0 0 * * 0'  # Weekly evidence collection

jobs:
  collect-evidence:
    runs-on: ubuntu-latest
    steps:
    - name: Collect SBOM
      run: |
        syft . -o spdx-json > evidence/sbom-$(date +%Y%m%d).json
        
    - name: Collect Vulnerability Scan
      run: |
        trivy fs --format json -o evidence/vuln-$(date +%Y%m%d).json .
        
    - name: Collect Infrastructure State
      run: |
        terraform show -json > evidence/tf-state-$(date +%Y%m%d).json
        
    - name: Sign Evidence
      run: |
        cosign sign-blob \
          --key env://COSIGN_PRIVATE_KEY \
          --output-signature evidence/signature-$(date +%Y%m%d).sig \
          evidence/*-$(date +%Y%m%d).json
        
    - name: Store Evidence
      run: |
        aws s3 sync evidence/ s3://compliance-bucket/$(date +%Y%m)/ \
          --storage-class GLACIER
```

## 50.7 Compliance Automation

Shift compliance left by automating checks in CI/CD rather than during annual audits.

### Compliance as Code

**Inspec/Chef Compliance**:
```ruby
# controls/container_security.rb
control 'container-1' do
  impact 1.0
  title 'Containers must not run as root'
  desc 'SOC 2 CC6.1: Logical access security'
  
  describe kubernetes.pod(name: 'app') do
    its('spec.containers.first.securityContext.runAsNonRoot') { should cmp true }
  end
end

control 'container-2' do
  impact 0.7
  title 'Read-only root filesystem'
  
  describe kubernetes.pod(name: 'app') do
    its('spec.containers.first.securityContext.readOnlyRootFilesystem') { should cmp true }
  end
end
```

**Compliance Check in CI**:
```yaml
- name: Compliance Check
  run: |
    inspec exec compliance-profile/ \
      --target kubernetes:// \
      --reporter cli junit:compliance-results.xml
    
    # Fail if any critical controls failed
    if grep -q 'failures="[1-9]' compliance-results.xml; then
      echo "Compliance check failed"
      exit 1
    fi
```

### Continuous Compliance Monitoring

**Falco Rules for Compliance**:
```yaml
# SOC 2: Detect unauthorized access to cardholder data
- rule: Unauthorized PHI Access
  desc: HIPAA - Detect access to PHI without proper role
  condition: >
    spawned_process and
    container.name contains "database" and
    (proc.name in (psql, mysql) or
     proc.cmdline contains "SELECT * FROM patients")
    and not user.name in (db_admin, backup_service)
  output: >
    HIPAA violation: Unauthorized PHI access
    user=%user.name command=%proc.cmdline
  priority: CRITICAL
```

**Continuous Validation**:
```yaml
# CronJob to validate compliance daily
apiVersion: batch/v1
kind: CronJob
metadata:
  name: compliance-validator
spec:
  schedule: "0 6 * * *"
  jobTemplate:
    spec:
      template:
        spec:
          containers:
          - name: validator
            image: compliance-tools:latest
            command:
            - /bin/sh
            - -c
            - |
              # Check all production deployments have required labels
              kubectl get deployments -n production -o json | \
                jq -r '.items[] | select(.metadata.labels["compliance.company.com/owner"] == null) | .metadata.name' | \
                while read deploy; do
                  echo "VIOLATION: Deployment $deploy missing owner label"
                  # Create incident ticket automatically
                done
              
              # Verify encryption in transit
              kubectl get ingresses -n production -o json | \
                jq '.items[] | select(.spec.tls == null) | .metadata.name' | \
                while read ing; do
                  echo "VIOLATION: Ingress $ing missing TLS"
                done
          restartPolicy: OnFailure
```

## 50.8 Reporting and Evidence

Compliance requires demonstrating control effectiveness to auditors through structured evidence packages.

### Automated Compliance Reports

**Monthly Compliance Dashboard**:
```yaml
apiVersion: batch/v1
kind: CronJob
metadata:
  name: compliance-report
spec:
  schedule: "0 0 1 * *"  # First of month
  jobTemplate:
    spec:
      template:
        spec:
          containers:
          - name: reporter
            image: compliance-reporter:latest
            env:
            - name: MONTH
              value: "$(date +%Y-%m)"
            command:
            - python
            - /app/generate_report.py
            - --month=$(MONTH)
            - --output=s3://compliance-reports/
            - --format=pdf,xlsx
          restartPolicy: OnFailure
```

**Report Contents**:
1. **Change Summary**: All deployments, who approved, what changed
2. **Vulnerability Status**: Open CVEs, remediation SLAs
3. **Access Reviews**: Who has production access, last login
4. **Policy Violations**: Failed compliance checks, remediation
5. **Backup Verification**: Last successful backup, test restore results
6. **Certificate Expiry**: TLS certs expiring in next 90 days

### Evidence Packages

**Audit Bundle Generation**:
```bash
#!/bin/bash
# generate-evidence.sh

TIMESTAMP=$(date +%Y%m%d-%H%M%S)
BUNDLE="evidence-${TIMESTAMP}.tar.gz"

# Collect evidence
mkdir -p evidence/${TIMESTAMP}

# 1. Pipeline logs
gh run list --limit 30 --json databaseId | \
  jq -r '.[].databaseId' | \
  while read run_id; do
    gh run view $run_id --log > evidence/${TIMESTAMP}/pipeline-${run_id}.log
  done

# 2. Kubernetes audit logs
kubectl logs -n kube-system --selector app=kube-apiserver --since=720h > \
  evidence/${TIMESTAMP}/k8s-audit.log

# 3. SBOMs
find . -name "sbom-*.json" -exec cp {} evidence/${TIMESTAMP}/ \;

# 4. Vulnerability scans
find . -name "vuln-*.json" -exec cp {} evidence/${TIMESTAMP}/ \;

# 5. Terraform state (sanitized)
terraform show -json | \
  jq 'del(.values.root_module.resources[].values.password)' > \
  evidence/${TIMESTAMP}/infrastructure-state.json

# 6. Sign bundle
tar -czf ${BUNDLE} evidence/${TIMESTAMP}
cosign sign-blob --key cosign.key ${BUNDLE}

# 7. Upload to immutable storage
aws s3 cp ${BUNDLE} s3://compliance-evidence-7year-worm/
aws s3 cp ${BUNDLE}.sig s3://compliance-evidence-7year-worm/

echo "Evidence bundle: ${BUNDLE}"
```

---

## Chapter Summary and Preview

This chapter established comprehensive compliance and governance frameworks for CI/CD pipelines, transforming regulatory requirements from manual audit burdens into automated, continuous controls. We examined major regulatory frameworks—**SOC 2** requiring logical access controls and audit trails, **PCI-DSS** mandating encryption and vulnerability management for payment data, **HIPAA** enforcing PHI access controls and audit logging, and **GDPR** requiring data protection by design and breach notification capabilities—and their specific technical implementations in Kubernetes environments.

**Audit trails** must capture every state change across the software supply chain: who committed code, who approved the pull request, what vulnerabilities were present in the build, who triggered deployment, and what configuration drift occurred. These logs require **immutable storage** with WORM (Write Once Read Many) policies and cryptographic integrity verification to satisfy legal hold requirements.

**Policy enforcement** through Open Policy Agent (OPA) Gatekeeper and Kyverno automates compliance checks, preventing non-compliant resources (privileged containers, missing labels, images from untrusted registries) from reaching production. **Approval workflows** enforce segregation of duties, ensuring that deployment to production requires independent validation from code authorship.

**Documentation** must be generated automatically from infrastructure-as-code, SBOMs, and pipeline logs rather than maintained manually, ensuring it reflects actual system state rather than intended design. **Evidence packages** for auditors should be generated on-demand, containing signed, tamper-evident bundles of all relevant artifacts.

**Key Takeaways:**
- Compliance is continuous, not a point-in-time audit; automate evidence collection and policy enforcement in CI/CD.
- Implement immutable audit logs with cryptographic signatures and WORM storage to satisfy legal and regulatory retention requirements.
- Use policy-as-code (OPA/Kyverno) to prevent non-compliant resources from reaching production rather than detecting violations post-deployment.
- Enforce segregation of duties through mandatory approval workflows for production deployments.
- Generate compliance documentation automatically from code and pipeline metadata; manual documentation inevitably drifts from reality.
- Maintain SBOMs and vulnerability scan results as part of the audit trail for software supply chain integrity.
- Encrypt etcd at rest using KMS providers; base64 encoding is not encryption, and etcd backups contain all cluster secrets.

**Compliance Culture:** Compliance is not security, but security enables compliance. Design systems where the secure path is the compliant path, automating evidence collection so that audits become demonstrations of continuous control rather than frantic searches for documentation.

**Next Chapter Preview:** Chapter 51: Scaling CI/CD Infrastructure addresses the operational challenges of growing CI/CD platforms serving multiple teams and thousands of pipelines. We will explore **horizontal scaling** of build agents through Kubernetes autoscaling and spot instances, **vertical scaling** for resource-intensive builds, **multi-region deployments** for disaster recovery and data residency compliance, **high availability** configurations for the CI/CD control plane, **disaster recovery** strategies including backup and restore of pipeline state, **capacity planning** methodologies to prevent queue bottlenecks, and **cost optimization** techniques including build caching, ephemeral environments, and rightsizing. We will examine how to maintain pipeline performance and reliability as organizational scale increases, ensuring that CI/CD infrastructure itself follows the same infrastructure-as-code and observability practices as the applications it builds.